## Análise Prescritiva - Prescriptive Analytics

- **Definição simples e clara:** Vai além da previsão e busca responder “o que deve ser feito” para atingir melhores resultados. Ela utiliza resultados de análises preditivas e técnicas de otimização para sugerir ações ótimas, considerando restrições e recursos disponíveis​. Em outras palavras, analisa cenários futuros possíveis e recomenda decisões ou estratégias para alcançar objetivos desejados​.

- **Aplicações práticas:** otimização de cadeias de suprimento (decidir quantos produtos fabricar/estocar ante previsão de demanda); precificação dinâmica (definir preços de passagens aéreas ou hotéis conforme demanda prevista); escalonamento de pessoal (quantos funcionários escalar em cada turno); roteirização de entregas (melhor rota para vários veículos); planejamento financeiro (como alocar orçamento entre projetos). Em saúde, por exemplo, pode sugerir qual tratamento dar a um paciente com base em previsões de eficácia; em marketing, recomenda quais ofertas personalizar.

- **Exemplo prático:** Uma loja prevê que terá alta demanda por um produto no próximo mês (análise preditiva). A análise prescritiva entra para dizer quantas unidades pedir ao fornecedor e quando, de modo a minimizar custos de estoque e evitar faltas. Outro exemplo: dada a previsão de tráfego de chamadas a um call center, determina-se quantos atendentes contratar em cada hora para manter qualidade.

- **Principais ferramentas/técnicas:** algoritmos de otimização (programação linear, programação inteira, programação dinâmica), simulações de Monte Carlo, heurísticas e algoritmos genéticos. Exemplos de softwares e bibliotecas: Google OR-Tools, PuLP ou Pyomo em Python, lpSolve em R, além de solvers comerciais como CPLEX e Gurobi. Ferramentas de simulação (Arena, Simul8) e técnicas de machine learning avançadas também podem ser usadas em prescrição.

---

### Exemplo prático de Análise Prescritiva em Python

**Empresa: Construtora LTDA**
**Faturamento total por ano:**
- Faturamento 2020: $ 8.500.000,00 
- Faturamento 2021: $ 7.450.000,00 
- Faturamento 2022: $ 6.800.000,00 
- Faturamento 2023: $ 8.000.500,00 
- Faturamento 2024: $ 8.500.000,00 

Custo médio total por ano
- Custo 2020: 82% do faturamento 
- Custo 2021: 78% do faturamento 
- Custo 2022: 85% do faturamento 
- Custo 2023: 77,5% do faturamento 
- Custo 2022: 76,9% do faturamento 

Distribuição Percentual Estimada dos Custos da Construção:
- Materiais: 50% dos custos total da obras. 
- Mão de Obra: 30% dos custos total da obras. 
- Projetos e Taxas: 5% dos custos total das obras. 
- Imprevistos (Reserva Técnica): 5% dos custos total estimado. 

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Dados fornecidos
faturamentos = {
    2020: 8_500_000,
    2021: 7_450_000,
    2022: 6_800_000,
    2023: 8_000_500,
    2024: 8_500_000
}

custos_pct = {
    2020: 0.82,
    2021: 0.78,
    2022: 0.85,
    2023: 0.775,
    2024: 0.769
}

# Calcular o orçamento de custos de 2024
receita_2024 = faturamentos[2024]
orcamento_custos_2024 = receita_2024 * custos_pct[2024]

# Benefícios por real investido em cada categoria (hipotéticos)
beneficios = np.array([1.05, 1.03, 1.10, 1.00])
categorias = ['Materiais', 'Mão de Obra', 'Projetos e Taxas', 'Imprevistos']

# Limites de porcentagem
lims_pct = {
    'Materiais': (0.45, 0.55),
    'Mão de Obra': (0.25, 0.35),
    'Projetos e Taxas': (0.04, 0.06),
    'Imprevistos': (0.03, 0.07)
}

# Converter limites para valores absolutos
bounds = [
    (lims_pct[cat][0] * orcamento_custos_2024, lims_pct[cat][1] * orcamento_custos_2024)
    for cat in categorias
]

# Preparar para linprog (minimização de -benefício para maximizar benefício)
c = -beneficios

# Restrição de igualdade: soma das alocações = orçamento de custos
A_eq = [np.ones(len(categorias))]
b_eq = [orcamento_custos_2024]

# Resolver LP
res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extrair e exibir resultados
alloc = res.x
retorno_esperado = beneficios * alloc

df = pd.DataFrame({
    'Categoria': categorias,
    'Alocado (R$)': alloc,
    'Percentual sobre custos (%)': alloc / orcamento_custos_2024 * 100,
    'Retorno Esperado (R$)': retorno_esperado
})

total = pd.DataFrame([{
    'Categoria': 'Total',
    'Alocado (R$)': orcamento_custos_2024,
    'Percentual sobre custos (%)': 100.0,
    'Retorno Esperado (R$)': retorno_esperado.sum()
}])

df = pd.concat([df, total], ignore_index=True)


display(df)


,Categoria,Alocado (R$),Percentual sobre custos (%),Retorno Esperado (R$)
0,Materiais,3595075.0,55.0,3774828.75
1,Mão de Obra,2287775.0,35.0,2356408.25
2,Projetos e Taxas,392190.0,6.0,431409.00
3,Imprevistos,261460.0,4.0,261460.00
4,Total,6536500.0,100.0,6824106.00


Aplicando otimização linear para recomendar a alocação ótima de orçamento de custos em 2025 para a Construtora LTDA, de modo a maximizar o retorno esperado de cada categoria de despesa, sujeito às restrições de percentuais:

1) Calculamos o orçamento de custos de 2024 (76,9 % de R$ 8.500.000 = R$ 6.536.500).

2) Definimos coeficientes de “benefício” por real investido em cada categoria (hipotéticos, só para ilustrar).

3) Estabelecemos limites mínimos e máximos de percentuais para Materiais (45–55 %), Mão de Obra (25–35 %), Projetos e Taxas (4–6 %) e Imprevistos (3–7 %).

4) Formulamos um problema de otimização linear (usando o método Highs do scipy.optimize.linprog) para maximizar o retorno total esperado.

5) O resultado mostra quanto alocar em cada categoria e o retorno correspondente:

| Categoria | Alocado (R$)	| % sobre custos | Retorno Esperado (R$)|
| --- | --- | --- | --- | 
| Materiais |	3.595.075,00 |	55,0 %	| 3.774.828,75 |
| Mão de Obra |	2.287.775,00 |	35,0 %	| 2.356.408,25 |
| Projetos e Taxas | 392.190,00	| 6,0 %	| 431.409,00 |
| Imprevistos |	261.460,00	| 4,0 %	| 261.460,00 |
| Total	| 6.536.500,00	| 100 % | 6.824.106,00 |

<br>
 A solução aloca o máximo permitido em Materiais (55%), Mão de Obra (35%) e Projetos e Taxas (6%), reduzindo Imprevistos ao mínimo permitido (4%), pois estes trazem menor retorno unitário. Assim, maximiza-se o retorno esperado de R$ 6 824 106 para o mesmo orçamento de custos.